In [ ]:
# default_exp callback.logtable

# LogTableMetricsCallback

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
%matplotlib inline

In [ ]:
#export
import copy
from itertools import zip_longest
from typing import List, Any, Dict, Callable
import time

from IPython.display import clear_output, display, HTML

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning.core import LightningModule
from pytorch_lightning.metrics import functional as FM
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.utilities import rank_zero_info

from isic.utils.core import format_time

C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [ ]:
#export
def text2html_table(items):
    # items = [[headers]]
    "Put the texts in `items` in an HTML table."
    html_code = f"""<table border="1" class="dataframe">\n"""
    html_code += f"""  <thead>\n    <tr style="text-align: left;">\n"""
    # headers in items[0]
    for i in items[0]: html_code += f"      <th>{i}</th>\n"
    html_code += f"    </tr>\n  </thead>\n  <tbody>\n"
    for line in items[1:]:
        html_code += "    <tr>\n"
        for i in line: html_code += f"      <td>{i}</td>\n"
        html_code += "    </tr>\n"
    html_code += "  </tbody>\n</table><p>"
    return html_code

In [ ]:
#export
#export
class LogTableMetricsCallback(Callback):
    def __init__(self, log_html=False):
        super().__init__()
        self.metrics = []
        self.headers = []
        self.rows = []
        self.log_html=log_html
        self.out = None

    def on_epoch_start(self, trainer, pl_module):
        self.start = time.time()

    def on_epoch_end(self, trainer, pl_module):
        t = format_time(time.time() - self.start)
        metrics = copy.copy(trainer.progress_bar_dict)
        if 'v_num' in metrics.keys():
            del metrics['v_num']
        if not self.headers:
            self.headers.append('epoch')
            for k in metrics.keys():
                if k == 'loss':
                    self.headers.append('train_loss')
                else:
                    self.headers.append(k)
            self.headers.append('time')
        row = []
        row.append(trainer.current_epoch + 1)
        for header in self.headers:
            if header == 'train_loss':
                header = 'loss'
            if header in metrics:
                row.append(metrics[header])
        row.append(t)
        row = [f"{v:.6f}" if isinstance(v, float) else str(v) for v in row]
        self.rows.append(row)
        if not self.out:
            self.out = display(HTML(text2html_table([self.headers, *self.rows])), display_id=True)
        self.out.update(HTML(text2html_table([self.headers, *self.rows])))
#         print(self.metrics)
#         print(trainer.progress_bar_dict)
#         print(trainer.optimizers)
#         print(trainer.lr_schedulers)

    def on_train_end(self, trainer, pl_module):
        if self.log_html:
            logger = getattr(trainer, logger)
            if isinstance(logger, pl.loggers.WandbLogger):
                import wandb
                trainer.logger.experiment.log({"train_table_log": wandb.Html(HTML(text2html_table([self.headers, *self.rows])))})

In [ ]:
from nbdev.export import *
notebook2script('cb_logtable.ipynb')

Converted cb_logtable.ipynb.
